# Lab 3.3.4: Medusa Speculative Decoding

**Module:** 3.3 - Model Deployment & Inference Engines  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐⭐⭐

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand the draft-verify paradigm of speculative decoding
- [ ] Know how Medusa adds prediction heads without a separate draft model
- [ ] Measure acceptance rates and speedups on different types of prompts
- [ ] Configure Medusa for optimal performance on DGX Spark

---

## 📚 Prerequisites

- Completed: Labs 3.3.1-3.3.3
- Knowledge of: Transformer architecture basics, attention mechanism
- Having: SGLang or vLLM with Medusa support

---

## 🌍 Real-World Context

**The Problem:** LLMs generate tokens one at a time (autoregressive). Each token requires a full forward pass through the model. This is inherently sequential and slow.

**The Opportunity:** GPUs are massively parallel but underutilized during autoregressive generation. Can we do useful parallel work?

**Speculative Decoding Solution:**
1. Quickly guess multiple future tokens (speculation)
2. Verify all guesses in parallel with the real model
3. Accept correct guesses, regenerate incorrect ones

**Real Impact:**
- 2-3x speedup for interactive chat
- No quality degradation (mathematically equivalent outputs)
- Especially effective for predictable outputs (code, structured data)

---

## 🧒 ELI5: What is Speculative Decoding?

> **Imagine you're playing a word guessing game with a very slow, very smart friend...**
>
> **OLD WAY:**
> - You: "What's the first word of your sentence?"
> - Friend: (thinks for 10 seconds) "The"
> - You: "What's the second word?"
> - Friend: (thinks for 10 seconds) "quick"
> - (This takes forever!)
>
> **SPECULATIVE DECODING WAY:**
> - You: "Is your sentence 'The quick brown fox'?"
> - Friend: (checks all 4 words at once in 10 seconds) "Yes to 'The quick brown', no to 'fox' - it's 'lazy'"
> - You got 3 words verified in the time of 1!
>
> **In AI terms:** A fast "draft" model (or heads) proposes multiple tokens. The slow "target" model verifies them in parallel. If the drafts are often correct, you get a big speedup!

---

## 🧒 ELI5: What is Medusa?

> **Traditional speculative decoding needs TWO models:**
> - A small, fast "draft" model to guess tokens
> - The real model to verify guesses
> - Problem: You need to store and run two models!
>
> **Medusa is smarter:**
> - Add extra "prediction heads" to the existing model
> - Head 1 predicts the next token (normal)
> - Head 2 predicts the token after that
> - Head 3 predicts 2 tokens ahead
> - And so on...
>
> **Like having multiple crystal balls** - each one sees a different distance into the future. All using the same model!
>
> **Named after Medusa** from Greek mythology who had multiple snake heads - each Medusa head predicts a different future token.

---

## Part 1: Understanding Medusa Architecture

In [ ]:
# Standard imports
import json
import os
import sys
import time
from pathlib import Path
from typing import Dict, List, Any, Optional
import warnings
warnings.filterwarnings('ignore')

# Third-party imports
import requests
import numpy as np

# Add scripts directory to path
scripts_path = Path("../scripts").resolve()
sys.path.insert(0, str(scripts_path))

try:
    from speculative_decoding import (
        MedusaConfig,
        SGLangSpeculativeClient,
        measure_acceptance_rate,
        get_optimal_speculation_config,
        format_speculation_report
    )
    print("✅ Speculative decoding utilities loaded")
except ImportError as e:
    print(f"⚠️ Could not load utilities: {e}")

print(f"📁 Scripts path: {scripts_path}")

In [ ]:
# Visualize Medusa architecture
print("""
📊 MEDUSA ARCHITECTURE
=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""

Traditional Transformer (one head):
───────────────────────────────────
                    ┌─────────────┐
  Hidden States ───►│  LM Head    │───► Token t+1
                    └─────────────┘

Medusa (multiple heads):
───────────────────────────────────
                    ┌─────────────┐
                ┌──►│  LM Head 0  │───► Token t+1 (original)
                │   └─────────────┘
                │   ┌─────────────┐
  Hidden    ────┼──►│ Medusa H1   │───► Token t+2 (speculation)
  States        │   └─────────────┘
                │   ┌─────────────┐
                ├──►│ Medusa H2   │───► Token t+3 (speculation)
                │   └─────────────┘
                │   ┌─────────────┐
                └──►│ Medusa H3   │───► Token t+4 (speculation)
                    └─────────────┘

Each Medusa head is a small MLP trained to predict future tokens.
Heads share the base model's hidden states - no extra forward passes!
""")

In [ ]:
# Explain the verification process
print("""
📊 MEDUSA VERIFICATION PROCESS
=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""=""

Step 1: SPECULATION
───────────────────
Given current tokens: ["The", "quick"]

Head 0 predicts: "brown"  (next token)
Head 1 predicts: "fox"    (t+2)
Head 2 predicts: "jumps"  (t+3)
Head 3 predicts: "over"   (t+4)

Candidate sequence: ["brown", "fox", "jumps", "over"]

Step 2: TREE-STYLE VERIFICATION
────────────────────────────────
Medusa uses "tree attention" to verify multiple candidates:

       "brown"
      /       \
   "fox"     "dog"     (multiple candidates per position)
   /   \     /    \
"jumps" "runs"  ...   (branch for each possibility)

The target model evaluates ALL branches in ONE forward pass!

Step 3: ACCEPTANCE
──────────────────
Compare Medusa predictions with target model:
- "brown" ✅ Accepted (matches target)
- "fox"   ✅ Accepted
- "jumps" ❌ Rejected (target says "leaps")

Result: Accept ["brown", "fox"], regenerate from "leaps"
Speedup: Generated 2 tokens in time of ~1!
""")

---

## Part 2: Medusa Configuration

In [ ]:
# Explore Medusa configuration options
print("📊 Medusa Configuration Options\n")

configs = {
    "Conservative": MedusaConfig(num_heads=3, max_speculation_length=3),
    "Balanced": MedusaConfig(num_heads=4, max_speculation_length=5),
    "Aggressive": MedusaConfig(num_heads=5, max_speculation_length=8),
}

print(f"{'Config':<15} {'Heads':<8} {'Max Spec':<10} {'Best For'}")
print("-" * 60)

for name, config in configs.items():
    if name == "Conservative":
        use_case = "Unpredictable outputs (creative writing)"
    elif name == "Balanced":
        use_case = "General-purpose chat"
    else:
        use_case = "Predictable outputs (code, lists)"
    
    print(f"{name:<15} {config.num_heads:<8} {config.max_speculation_length:<10} {use_case}")

In [ ]:
# Show recommended configs for different prompt types
print("\n📊 Optimal Configuration by Prompt Type\n")

prompt_types = ["predictable", "code", "general", "creative"]

for ptype in prompt_types:
    config = get_optimal_speculation_config(ptype)
    print(f"\n{ptype.upper()}:")
    print(f"  Method: {config['method']}")
    print(f"  Heads: {config['num_heads']}")
    print(f"  Max speculation: {config['max_speculation_length']}")
    print(f"  Expected acceptance: {config['expected_acceptance_rate']:.0%}")
    print(f"  Expected speedup: {config['expected_speedup']:.1f}x")
    print(f"  Notes: {config['notes']}")

---

## Part 3: Measuring Acceptance Rates

The key metric for speculative decoding is the **acceptance rate** - what percentage of speculated tokens are correct?

In [ ]:
# Load benchmark prompts
data_path = Path("../data/benchmark_prompts.json")

if data_path.exists():
    with open(data_path) as f:
        all_prompts = json.load(f)
    
    # Get speculative decoding test prompts
    spec_prompts = all_prompts.get("speculative_decoding_test", [])
    print(f"📝 Loaded {len(spec_prompts)} speculative decoding test prompts")
    
    for p in spec_prompts:
        print(f"   • [{p['category']}] {p['text'][:50]}...")
else:
    print("⚠️ Benchmark prompts not found. Creating test prompts...")
    spec_prompts = [
        {"text": "Count from 1 to 10.", "category": "predictable"},
        {"text": "Write a creative story.", "category": "creative"},
    ]

In [ ]:
# Define test prompts by category
PREDICTABLE_PROMPTS = [
    "Count from 1 to 20, one number per line.",
    "List the days of the week.",
    "Write the alphabet from A to Z.",
    "List the months of the year.",
    "Count by 5s from 5 to 50.",
]

CODE_PROMPTS = [
    "Write a Python function to calculate factorial.",
    "Write a Python class for a linked list node.",
    "Write a Python function to check if a string is a palindrome.",
    "Write a Python function to merge two sorted lists.",
]

CREATIVE_PROMPTS = [
    "Write a haiku about artificial intelligence.",
    "Create a short story about a robot who learns to paint.",
    "Describe an alien world in vivid detail.",
    "Write a poem about the ocean at sunset.",
]

print(f"📝 Test prompts prepared:")
print(f"   Predictable: {len(PREDICTABLE_PROMPTS)}")
print(f"   Code: {len(CODE_PROMPTS)}")
print(f"   Creative: {len(CREATIVE_PROMPTS)}")

In [ ]:
# Check if server with Medusa is available
SGLANG_URL = "http://localhost:30000"

def check_medusa_server(url: str) -> bool:
    """Check if a server with Medusa support is available."""
    try:
        response = requests.get(f"{url}/v1/models", timeout=5)
        if response.status_code == 200:
            print(f"✅ Server available at {url}")
            # Check for Medusa-specific endpoints if available
            return True
    except requests.exceptions.ConnectionError:
        print(f"❌ No server at {url}")
        print("\n📝 To run Medusa with SGLang:")
        print("   python -m sglang.launch_server \\")
        print("       --model-path lmms-lab/llama-3.1-8b-medusa \\")
        print("       --port 30000 \\")
        print("       --speculative-algorithm medusa")
    return False

medusa_available = check_medusa_server(SGLANG_URL)

In [ ]:
def benchmark_acceptance_rate(
    url: str,
    prompts: List[str],
    category: str,
    max_tokens: int = 100
) -> Dict[str, Any]:
    """
    Benchmark speculative decoding on a set of prompts.
    
    Returns acceptance rate and timing statistics.
    """
    print(f"\n🧪 Benchmarking {category} prompts ({len(prompts)} total)...")
    
    results = []
    
    for i, prompt in enumerate(prompts):
        start_time = time.perf_counter()
        
        try:
            response = requests.post(
                f"{url}/v1/chat/completions",
                json={
                    "model": "default",
                    "messages": [{"role": "user", "content": prompt}],
                    "max_tokens": max_tokens,
                    "temperature": 0.7
                },
                timeout=60
            )
            
            elapsed = time.perf_counter() - start_time
            
            if response.status_code == 200:
                data = response.json()
                usage = data.get("usage", {})
                tokens = usage.get("completion_tokens", 0)
                
                # Get speculation stats if available
                spec_stats = data.get("speculation_stats", {})
                accepted = spec_stats.get("accepted", tokens)
                rejected = spec_stats.get("rejected", 0)
                
                result = {
                    "prompt": prompt,
                    "tokens": tokens,
                    "time": elapsed,
                    "tokens_per_second": tokens / elapsed if elapsed > 0 else 0,
                    "accepted": accepted,
                    "rejected": rejected,
                    "acceptance_rate": accepted / (accepted + rejected) if (accepted + rejected) > 0 else 1.0,
                    "success": True
                }
                results.append(result)
                print(f"   [{i+1}] {tokens} tokens, {result['tokens_per_second']:.0f} tok/s")
            else:
                print(f"   [{i+1}] ❌ Error: {response.status_code}")
                
        except Exception as e:
            print(f"   [{i+1}] ❌ Error: {e}")
    
    # Compute aggregate stats
    if results:
        return {
            "category": category,
            "num_prompts": len(prompts),
            "successful": len(results),
            "avg_tokens_per_second": np.mean([r["tokens_per_second"] for r in results]),
            "avg_acceptance_rate": np.mean([r["acceptance_rate"] for r in results]),
            "total_tokens": sum(r["tokens"] for r in results),
            "total_time": sum(r["time"] for r in results),
        }
    else:
        return {"category": category, "error": "No successful requests"}

In [ ]:
# Run benchmarks if server is available
if medusa_available:
    all_category_results = {}
    
    # Benchmark each category
    categories = [
        ("predictable", PREDICTABLE_PROMPTS),
        ("code", CODE_PROMPTS),
        ("creative", CREATIVE_PROMPTS),
    ]
    
    for category, prompts in categories:
        result = benchmark_acceptance_rate(
            url=SGLANG_URL,
            prompts=prompts,
            category=category,
            max_tokens=100
        )
        all_category_results[category] = result
    
    # Print summary
    print("\n" + "="*60)
    print("📊 ACCEPTANCE RATE BY CATEGORY")
    print("="*60)
    print(f"\n{'Category':<15} {'Acceptance':<12} {'Speed (tok/s)':<15} {'Expected'}")
    print("-" * 55)
    
    expected_rates = {"predictable": 0.8, "code": 0.65, "creative": 0.35}
    
    for category, result in all_category_results.items():
        if "error" not in result:
            expected = expected_rates.get(category, 0.5)
            print(f"{category:<15} {result['avg_acceptance_rate']:.1%}{'':>5} "
                  f"{result['avg_tokens_per_second']:.0f}{'':>10} ~{expected:.0%}")
        else:
            print(f"{category:<15} Error")
else:
    print("\n⚠️ Server not available.")
    print("\n📊 Expected results with Medusa:")
    print("   Predictable prompts: ~80% acceptance, 2.5x speedup")
    print("   Code prompts: ~65% acceptance, 2.0x speedup")
    print("   Creative prompts: ~35% acceptance, 1.3x speedup")

### 🔍 What Just Happened?

We measured acceptance rates across different prompt types:

1. **Predictable prompts** (counting, lists) → **High acceptance (~80%)**
   - Output is largely deterministic
   - Medusa heads can easily predict next tokens
   - Best speedup (2-3x)

2. **Code prompts** → **Medium acceptance (~65%)**
   - Syntax is predictable (brackets, keywords)
   - Logic is less predictable
   - Good speedup (1.5-2x)

3. **Creative prompts** → **Lower acceptance (~35%)**
   - Output is intentionally unpredictable
   - Many valid next tokens possible
   - Modest speedup (1.1-1.4x)

---

## Part 4: Speedup Analysis

Let's understand the relationship between acceptance rate and speedup.

In [ ]:
def calculate_theoretical_speedup(
    acceptance_rate: float,
    num_heads: int,
    overhead: float = 0.1  # Overhead from tree verification
) -> float:
    """
    Calculate theoretical speedup from speculative decoding.
    
    Speedup ≈ 1 / (1 - acceptance_rate * (num_heads - 1) / num_heads) - overhead
    
    Simplified: Expected tokens accepted per step / cost per step
    """
    # Expected tokens per speculation round
    expected_accepted = 1 + acceptance_rate * (num_heads - 1)
    
    # Cost is approximately 1 + small overhead for verification
    cost = 1 + overhead
    
    return expected_accepted / cost


# Visualize speedup vs acceptance rate
print("📊 Theoretical Speedup vs Acceptance Rate\n")
print(f"{'Acceptance':<12} {'3 Heads':<10} {'4 Heads':<10} {'5 Heads':<10}")
print("-" * 45)

for acc_rate in [0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    speedups = [
        calculate_theoretical_speedup(acc_rate, 3),
        calculate_theoretical_speedup(acc_rate, 4),
        calculate_theoretical_speedup(acc_rate, 5),
    ]
    print(f"{acc_rate:.0%}{'':>6} {speedups[0]:.2f}x{'':>5} {speedups[1]:.2f}x{'':>5} {speedups[2]:.2f}x")

In [ ]:
# Visualize the relationship
try:
    import matplotlib.pyplot as plt
    
    acceptance_rates = np.linspace(0.1, 0.95, 50)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    for num_heads in [3, 4, 5]:
        speedups = [calculate_theoretical_speedup(a, num_heads) for a in acceptance_rates]
        ax.plot(acceptance_rates * 100, speedups, label=f'{num_heads} Medusa heads', linewidth=2)
    
    # Mark typical acceptance rates
    typical_points = {
        "Predictable": 0.80,
        "Code": 0.65,
        "General": 0.50,
        "Creative": 0.35
    }
    
    for label, rate in typical_points.items():
        speedup = calculate_theoretical_speedup(rate, 4)
        ax.scatter([rate * 100], [speedup], s=100, zorder=5)
        ax.annotate(label, (rate * 100, speedup), 
                   xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    ax.set_xlabel('Acceptance Rate (%)', fontsize=12)
    ax.set_ylabel('Speedup Factor (x)', fontsize=12)
    ax.set_title('Medusa Speedup vs Acceptance Rate', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='No speedup')
    
    plt.tight_layout()
    plt.savefig('medusa_speedup.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n📈 Chart saved to medusa_speedup.png")
    
except ImportError:
    print("⚠️ matplotlib not available")

---

## ⚠️ Common Mistakes

### Mistake 1: Too Many Medusa Heads

```python
# ❌ Wrong - Too many heads with low acceptance rate
config = MedusaConfig(num_heads=8, max_speculation_length=10)
# On creative prompts, most speculations are wrong → wasted compute

# ✅ Right - Match heads to prompt type
config = MedusaConfig(num_heads=3, max_speculation_length=4)  # For creative
config = MedusaConfig(num_heads=5, max_speculation_length=8)  # For predictable
```

### Mistake 2: Expecting Speedup on All Prompts

```python
# ❌ Wrong - Same config for everything
# Creative writing with aggressive speculation = minimal speedup

# ✅ Right - Adaptive configuration
if prompt_is_structured(prompt):
    use_aggressive_speculation()
else:
    use_conservative_speculation()  # Or disable speculation
```

### Mistake 3: Not Accounting for Temperature

```python
# ❌ Wrong - High temperature = unpredictable output
temperature = 1.5  # Very random → low acceptance rate

# ✅ Right - Lower temperature improves acceptance
temperature = 0.7  # Balanced
temperature = 0.0  # Greedy (highest acceptance, but less diverse)
```

---

## ✋ Try It Yourself

### Exercise 1: Find the Optimal Number of Heads

For a specific prompt type, find the optimal number of Medusa heads.

In [ ]:
# Exercise 1: Your code here
# Test different numbers of heads (if you can configure your server)
# and measure actual speedup vs theoretical

# Heads to test: [2, 3, 4, 5, 6]
# For each, measure:
# - Actual tokens/second
# - Acceptance rate
# - Compare to baseline (1 head)

# TODO: Implement the experiment

### Exercise 2: Analyze Temperature Impact

Test how temperature affects acceptance rate.

In [ ]:
# Exercise 2: Your code here
# Test temperatures: [0.0, 0.3, 0.5, 0.7, 1.0, 1.5]
# For each, measure acceptance rate on the same prompts
# Plot temperature vs acceptance rate

# TODO: Implement the experiment

---

## 🎉 Checkpoint

You've learned:
- ✅ How speculative decoding uses draft-verify to speed up generation
- ✅ How Medusa adds prediction heads without needing a separate model
- ✅ The relationship between acceptance rate and speedup
- ✅ How to configure Medusa for different prompt types

---

## 🚀 Challenge (Optional)

**Build an Adaptive Speculation System**

Create a system that:
1. Classifies incoming prompts (predictable vs creative)
2. Dynamically adjusts speculation depth
3. Monitors acceptance rate in real-time
4. Falls back to normal decoding when acceptance is too low

---

## 📖 Further Reading

- [Medusa Paper: Simple Framework for Fast LLM Inference](https://arxiv.org/abs/2401.10774)
- [Speculative Decoding Survey](https://arxiv.org/abs/2401.07851)
- [Medusa GitHub Repository](https://github.com/FasterDecoding/Medusa)
- [SGLang Speculative Decoding Docs](https://sgl-project.github.io/)

---

## 🧹 Cleanup

In [ ]:
# Cleanup
import gc
gc.collect()

print("✅ Cleanup complete!")